In [532]:
from itertools import tee, combinations
from functools import wraps

#### generators

In [533]:
def count(start=0, step=1, stop=10):
    n = start
    while n <= stop:
        yield n
        n += step

for x in count(10, 2.5, 20):
    print(x)

10
12.5
15.0
17.5
20.0


In [534]:
def generator():
    yield 'this is a generator'
    return 'returning from a generator'

g = generator()

In [535]:
next(g)

'this is a generator'

In [536]:
# next(g)

In [537]:
generator = (x ** 2 for x in range(4))

for x in generator:
   print(x)


0
1
4
9


In [538]:
class Count(object):
    def __init__(self, start=0, step=1, stop=10):
        self.n = start
        self.step = step
        self.stop = stop

    def __iter__(self):
        return self

    def __next__(self):
        n = self.n
        if n > self.stop:
            raise StopIteration()
        self.n += self.step
        return n


xcount = Count(10, 2.5, 20)
type(xcount)

__main__.Count

In [539]:
for x in xcount:
    print(x)

10
12.5
15.0
17.5
20.0


#### good and bad

In [540]:
def generator():
    print('Before 1')
    yield 1
    print('After 1')
    print('Before 2')
    yield 2
    print('After 2')
    print('Before 3')
    yield 3
    print('After 3')

In [541]:
g = generator()

In [542]:
print('Got %d' % next(g))

Before 1
Got 1


In [543]:
print('Got %d' % next(g))

After 1
Before 2
Got 2


In [544]:
print('Got %d' % next(g))

After 2
Before 3
Got 3


#### pipeline

In [545]:
# Prepare lines.txt
with open('lines.txt', 'w') as file:
    written = file.write('''
spam
eggs
spam spam
eggs eggs
spem spem
spam spam spam
spem spem
eggs eggs eggs
    '''.strip())

In [546]:
def cat(filename):
    for line in open(filename):
        yield line.rstrip()


def grep(sequence, search):
    for line in sequence:
        if search in line:
            yield line


def replace(sequence, search, replace):
    for line in sequence:
        yield line.replace(search, replace)

In [547]:
lines = cat('lines.txt')
lines

<generator object cat at 0x7f33a6e6f780>

In [548]:
spam_lines = grep(lines, 'spam')
spam_lines

<generator object grep at 0x7f33a6d02420>

In [549]:
bacon_lines = replace(spam_lines, 'spam', 'becon')
bacon_lines

<generator object replace at 0x7f33a6d31030>

In [550]:
for line in bacon_lines:
    print(line)

becon
becon becon
becon becon becon


In [551]:
# Or the one-line version, fits within 78 characters:
for line in replace(grep(cat('lines.txt'), 'spem'),
                    'spem', 'ahihi'):
    print(line)

ahihi ahihi
ahihi ahihi


#### tee

In [552]:

def spam_and_eggs():
    yield 'spam'
    yield 'eggs'

a, b = tee(spam_and_eggs())

In [553]:
next(a)

'spam'

In [554]:
next(a)

'eggs'

In [555]:
next(b)

'spam'

In [556]:
next(b)

'eggs'

In [557]:
# next(b)

#### generator from generator

In [558]:
def powerset(sequence):
    for size in range(len(sequence) + 1):
        for item in combinations(sequence, size):
            yield item


for result in powerset('abc'):
    print(result)

()
('a',)
('b',)
('c',)
('a', 'b')
('a', 'c')
('b', 'c')
('a', 'b', 'c')


In [559]:
def powerset(sequence):
    for size in range(len(sequence) + 1):
        # yield from syntax is new in Python 3.3
        yield from combinations(sequence, size)


for result in powerset('abc'):
    print(result)

()
('a',)
('b',)
('c',)
('a', 'b')
('a', 'c')
('b', 'c')
('a', 'b', 'c')


In [560]:
def flatten(sequence):
    for item in sequence:
        try:
            yield from flatten(item)
        except TypeError:
            yield item

list(flatten([1, [2, [3, [4, 5], 6], 7], 8]))

[1, 2, 3, 4, 5, 6, 7, 8]

#### context managers

In [561]:
import datetime
from contextlib import contextmanager, redirect_stdout, ExitStack


# Context manager that shows how long a context was active
@contextmanager
def timer(name):
    start_time = datetime.datetime.now()
    yield
    stop_time = datetime.datetime.now()
    print('%s took %s' % (name, stop_time - start_time))


# The write to log function writes all stdout (regular print data) to
# a file. The redirect_stdout context wrapper
# temporarily redirects standard output to a given file handle, in
# this case the file we just opened for writing.
@contextmanager
def write_to_log(name):
    with open('%s.txt' % name, 'w') as fh:
        with redirect_stdout(fh):
            with timer(name):
                yield

In [562]:
# @contextmanager
# def write_to_log(name):
#     with ExitStack() as stack:
#         file = stack.enter_context(open('stdout.txt', 'w'))
#         stack.enter_context(redirect_stdout(file))
#         stack.enter_context(timer(name))
#         yield

In [563]:
# Use the context manager as a decorator
@write_to_log('some function')
def some_function():
    print('This function takes a bit of time to execute')
    print('Do more')


some_function()

In [564]:
with ExitStack() as stack:
    spam_fh = stack.enter_context(open('spam.txt', 'w'))
    eggs_fh = stack.enter_context(open('eggs.txt', 'w'))
    spam_bytes_written = spam_fh.write('writing to spam')
    eggs_bytes_written = eggs_fh.write('writing to eggs')

    # Move the contexts to a new ExitStack and store the
    # close method
    close_handlers = stack.pop_all().close

spam_bytes_written = spam_fh.write('still writing to spam')
eggs_bytes_written = eggs_fh.write('still writing to eggs')

In [565]:
spam_bytes_written

21

In [566]:
eggs_bytes_written

21

In [567]:
# # # After closing we can't write anymore
# close_handlers()
# spam_bytes_written = spam_fh.write('cant write anymore')
# # spam_bytes_written

#### coroutine

In [568]:
def coroutine(function):
    @wraps(function)
    def _coroutine(*args, **kwargs):
        active_coroutine = function(*args, **kwargs)
        next(active_coroutine)
        return active_coroutine
    return _coroutine

In [569]:
@coroutine
def spam():
    while True:
        print('Waiting for yield...')
        value = yield
        print('spam received: %s' % value)


generator = spam()

Waiting for yield...


In [570]:
generator.send('a')


spam received: a
Waiting for yield...


In [571]:
generator.send('b')


spam received: b
Waiting for yield...


In [572]:
generator.send('c')


spam received: c
Waiting for yield...


In [573]:
@coroutine
def simple_coroutine():
    print('Setting up the coroutine')
    try:
        while True:
            item = yield
            print('Got item: %r' % item)
    except GeneratorExit:
        print('Normal exit')
    except Exception as e:
        print('Exception exit: %r' % e)
        raise
    finally:
        print('Any exit')

In [574]:
print('Creating simple coroutine')
active_coroutine = simple_coroutine()
print()

print('Sending spam')
active_coroutine.send('spam')
print()

print('Close the coroutine')
active_coroutine.close()
print()

print('Creating simple coroutine')
active_coroutine = simple_coroutine()
print()

print('Sending eggs')
active_coroutine.send('eggs')
print()

print('Throwing runtime error')
try:
    active_coroutine.throw(RuntimeError, 'Oops...')
except RuntimeError as exception:
    print(exception.args)
    pass


Creating simple coroutine
Setting up the coroutine

Sending spam
Got item: 'spam'

Close the coroutine
Normal exit
Any exit

Creating simple coroutine
Setting up the coroutine

Sending eggs
Got item: 'eggs'

Throwing runtime error
Exception exit: RuntimeError('Oops...')
Any exit
('Oops...',)


#### bidirectional pipeline

In [575]:
@coroutine
def replace(search, replace):
    while True:
        item = yield
        print(item.replace(search, replace))



In [576]:
spam_replace = replace('spam', 'sida')
for line in open('lines.txt'):
    spam_replace.send(line.rstrip())

sida
eggs
sida sida
eggs eggs
spem spem
sida sida sida
spem spem
eggs eggs eggs


In [577]:
@coroutine
def replace(search, replace):
    while True:
        item = yield
        # not print as above
        yield item.replace(search, replace)

In [578]:
spam_replace = replace('spam', 'sida')
spam_replace.send('spam')

'sida'

In [579]:
spam_replace.send('spam spam')

In [580]:
spam_replace.send('spam spam spam')

'sida sida sida'

In [581]:
@coroutine
def replace(search, replace):
    item = yield
    while True:
        item = yield item.replace(search, replace)

In [582]:
spam_replace = replace('spam', 'sidasida')

In [583]:
spam_replace.send('spam')

'sidasida'

In [584]:
spam_replace.send('spam spam')

'sidasida sidasida'

In [585]:
spam_replace.send('spam spam spam')

'sidasida sidasida sidasida'

In [586]:
# Grep sends all matching items to the target
@coroutine
def grep(target, pattern):
    while True:
        item = yield
        if pattern in item:
            # target.send((yield))
            target.send(item)

In [587]:
# Replace does a search and replace on the items and sends it to
# the target once it’s done
@coroutine
def replace(target, search, replace):
    while True:
        target.send((yield).replace(search, replace))

In [588]:
# Print will print the items using the provided formatstring
@coroutine
def print_(formatstring):
    while True:
        print(formatstring % (yield))

In [589]:
# Tee multiplexes the items to multiple targets
@coroutine
def tee(*targets):
    while True:
        item = yield
        for target in targets:
            # target.send((yield))
            target.send(item)

In [590]:
# Because we wrap the results we need to work backwards from the
# inner layer to the outer layer.

# First, create a printer for the items:
printer = print_('%s')

In [591]:
# Create replacers that send the output to the printer
replacer_spam = replace(printer, 'spam', 'bacon')
replacer_spam

<generator object replace at 0x7f33a6d31c60>

In [592]:
replacer_eggs = replace(printer, 'spam spam', 'sausage')
replacer_eggs

<generator object replace at 0x7f33a6d317b0>

In [593]:
# Create a tee to send the input to both the spam and the eggs
# replacers
branch = tee(replacer_spam, replacer_eggs)
branch

<generator object tee at 0x7f33a6d01d20>

In [594]:
# Send all items containing spam to the tee command
grepper = grep(branch, 'spam')
grepper

<generator object grep at 0x7f33a6d02880>

In [595]:
# Send the data to the grepper for all the processing
for line in open('lines.txt'):
    grepper.send(line.rstrip())

bacon
spam
bacon bacon
sausage
bacon bacon bacon
sausage spam


#### can bring that matching algorithm to the next level???

#### states

In [596]:
@coroutine
def average():
    count = 1	
    total = yield
    while True:
        total += yield total / count
        count += 1

averager = average()

In [597]:
averager.send(20)

20.0

In [598]:
averager.send(10)

15.0

In [599]:
averager.send(15)

15.0

In [600]:
averager.send(-25)


5.0

In [601]:
@coroutine
def print_(formatstring):
    while True:
        print(formatstring % (yield))

In [602]:
# another way to write average
@coroutine
def average(target):
    count = 0
    total = 0
    while True:
        count += 1
        total += yield
        target.send(total / count)

In [603]:
printer = print_('%.1f')
averager = average(printer)
averager.send(20)

20.0


In [604]:
averager.send(10)

15.0


In [605]:
averager.send(15)

15.0


In [606]:
averager.send(-25)

5.0


In [631]:
@coroutine
def print_(formatstring):
    while True:
        print(formatstring % (yield))

In [632]:
@coroutine
def groupby():
    # Fetch the first key and value and initialize the state
    # variables
    key, value = yield
    old_key, values = key, []
    while True:
        # Store the previous value so we can store it in the list
        old_value = value
        if key == old_key:
            key, value = yield
        else:
            key, value = yield old_key, values
            old_key, values = key, []
        values.append(old_value)

In [633]:
grouper = groupby()
grouper

<generator object groupby at 0x7f33a6d322f0>

In [634]:
grouper.send(('a', 1))
grouper.send(('a', 2))
grouper.send(('a', 3))
grouper.send((None, None))

('a', [1, 2, 3])

In [635]:
grouper.send(('b', 1))
grouper.send(('b', 2))
grouper.send(('a', 1))

('b', [None, 1, 2])

In [636]:
grouper.send(('a', 2))

In [637]:
grouper.send((None, None))

('a', [1, 2])

In [639]:
@coroutine
def groupby(target):
    old_key = None
    while True:
        key, value = yield
        if old_key != key:
            # A different key means a new group so send the previous group and restart the cycle.
            if old_key and values:
                target.send((old_key, values))
            values = []
            old_key = key
        values.append(value)

In [640]:
grouper = groupby(print_('group: %s, values: %s'))

In [641]:
grouper.send(('a', 1))
grouper.send(('a', 2))
grouper.send(('a', 3))
grouper.send(('b', 1))

group: a, values: [1, 2, 3]


In [642]:
grouper.send(('b', 2))
grouper.send(('a', 1))

group: b, values: [1, 2]


In [643]:
grouper.send(('a', 2))
grouper.send((None, None))

group: a, values: [1, 2]
